In [5]:

import numpy as np
import pandas as pd 
import seaborn as sns  

import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from sklearn.metrics import mean_squared_error
# from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

import time	
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline 
from pylab import rcParams

rcParams['figure.dpi'] = 120
rcParams['savefig.dpi'] = 120 
rcParams['font.size'] = 20


from tqdm import tqdm

In [6]:
df = pd.read_csv('../datasets/sales_train_validation.csv')
print(df.shape)
df.head()

(30490, 1919)


,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


In [7]:
submission = pd.read_csv('../datasets/sample_submission.csv')
print(submission.shape)
submission.head()

(60980, 29)


,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,HOBBIES_1_005_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
price_df = pd.read_csv('../datasets/sell_prices.csv')
print(price_df.shape)
price_df.head()

(6841121, 4)


,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


In [9]:
cal_df = pd.read_csv('../datasets/calendar.csv')
print(cal_df.shape)
cal_df.head(5)

(1969, 14)


,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [10]:
cal_df["d"]=cal_df["d"].apply(lambda x: int(x.split("_")[1])) # d_1 to 1
cal_df.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,5,NaN,NaN,NaN,NaN,1,0,1


In [11]:
price_df["id"] = price_df["item_id"] + "_" + price_df["store_id"] + "_validation"

In [12]:
price_df.head()

,store_id,item_id,wm_yr_wk,sell_price,id
0,CA_1,HOBBIES_1_001,11325,9.58,HOBBIES_1_001_CA_1_validation
1,CA_1,HOBBIES_1_001,11326,9.58,HOBBIES_1_001_CA_1_validation
2,CA_1,HOBBIES_1_001,11327,8.26,HOBBIES_1_001_CA_1_validation
3,CA_1,HOBBIES_1_001,11328,8.26,HOBBIES_1_001_CA_1_validation
4,CA_1,HOBBIES_1_001,11329,8.26,HOBBIES_1_001_CA_1_validation


In [13]:
days_cols = df.columns[df.columns.str.contains("d_")].values
df_days = df[days_cols]
df_days

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,0,0,0,0,0,0,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,0,0,0,0,0,0,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,0,0,0,0,0,0,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,0,0,2,2,0,3,1,4,1,0,...,2,0,0,0,0,0,1,0,0,1
30486,0,0,0,0,0,5,0,1,1,3,...,0,0,0,0,0,0,0,0,1,0
30487,0,6,0,2,2,4,1,8,5,2,...,2,1,0,2,0,1,0,0,1,0
30488,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,3,1,3


In [14]:
unit_sales_df =pd.DataFrame()

for day in tqdm(range(1858, 1886)):
    wk_id = list(cal_df[cal_df["d"]==day]["wm_yr_wk"])[0]
#     print("wk_id",wk_id)
    wk_price_df = price_df[price_df["wm_yr_wk"]==wk_id]
#     print("wk_price_df",wk_price_df)
    df = df.merge(wk_price_df[["sell_price", "id"]], on=["id"], how='inner')
#     df["unit_sales_" + str(day)] = df["sell_price"] * df["d_" + str(day)]
    unit_sales_df["unit_sales_" + str(day)] = df["sell_price"] * df["d_" + str(day)]
    df.drop(columns=["sell_price"], inplace=True)

100%|██████████| 28/28 [00:21<00:00,  1.29it/s]


In [15]:
df

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,2,0,0,0,0,0,1,0,0,1
30486,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
30487,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,2,1,0,2,0,1,0,0,1,0
30488,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,1,0,0,1,0,3,1,3


In [16]:
unit_sales_df

,unit_sales_1858,unit_sales_1859,unit_sales_1860,unit_sales_1861,unit_sales_1862,unit_sales_1863,unit_sales_1864,unit_sales_1865,unit_sales_1866,unit_sales_1867,...,unit_sales_1876,unit_sales_1877,unit_sales_1878,unit_sales_1879,unit_sales_1880,unit_sales_1881,unit_sales_1882,unit_sales_1883,unit_sales_1884,unit_sales_1885
0,0.00,16.52,0.00,8.26,8.26,0.00,8.26,0.00,0.00,8.26,...,24.78,8.26,24.78,8.26,16.52,16.52,0.00,8.26,8.26,8.26
1,0.00,0.00,0.00,3.97,0.00,0.00,3.97,3.97,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,3.97,3.97,3.97,3.97
2,0.00,0.00,0.00,0.00,0.00,17.82,2.97,2.97,5.94,0.00,...,2.97,0.00,0.00,0.00,0.00,0.00,0.00,2.97,2.97,0.00
3,0.00,0.00,0.00,4.64,0.00,4.64,23.20,13.92,4.64,0.00,...,18.56,9.28,4.64,18.56,4.64,13.92,23.20,0.00,27.84,27.84
4,2.88,5.76,2.88,0.00,0.00,2.88,0.00,5.76,2.88,2.88,...,8.64,5.76,5.76,5.76,8.64,2.88,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,0.00,0.00,5.76,0.00,0.00,5.96,2.98,2.98,0.00,2.98,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
30486,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
30487,3.98,7.96,7.96,7.96,7.96,7.96,7.96,3.98,3.98,7.96,...,7.96,0.00,0.00,7.96,0.00,0.00,0.00,7.96,0.00,3.98
30488,2.56,2.56,2.56,2.56,0.00,2.56,2.56,0.00,0.00,0.00,...,1.28,1.28,1.28,1.28,0.00,2.56,1.28,1.28,2.56,5.12


In [17]:
df["dollar_sales"] = unit_sales_df.sum(axis=1)

In [18]:
print(df.shape)
print(df.columns)
df.head()

(30490, 1920)
Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd_1',
       'd_2', 'd_3', 'd_4',
       ...
       'd_1905', 'd_1906', 'd_1907', 'd_1908', 'd_1909', 'd_1910', 'd_1911',
       'd_1912', 'd_1913', 'dollar_sales'],
      dtype='object', length=1920)


,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913,dollar_sales
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,3,0,1,1,1,3,0,1,1,231.28
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,1,0,0,0,0,35.73
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,2,1,1,1,0,1,1,1,44.55
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,5,4,1,0,1,3,7,2,245.92
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,1,1,2,2,2,4,89.28


In [19]:
df["weight"] = df["dollar_sales"] / df["dollar_sales"].sum()

In [20]:
print(df.shape)
print(df.columns)
df.head()

(30490, 1921)
Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd_1',
       'd_2', 'd_3', 'd_4',
       ...
       'd_1906', 'd_1907', 'd_1908', 'd_1909', 'd_1910', 'd_1911', 'd_1912',
       'd_1913', 'dollar_sales', 'weight'],
      dtype='object', length=1921)


,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913,dollar_sales,weight
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,1,1,3,0,1,1,231.28,0.000063
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,1,0,0,0,0,35.73,0.000010
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,1,0,1,1,1,44.55,0.000012
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,5,4,1,0,1,3,7,2,245.92,0.000067
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,1,1,2,2,2,4,89.28,0.000024


In [21]:
df.drop(columns=["dollar_sales"], inplace=True)
print(df.shape)
print(df.columns)
df.head()

(30490, 1920)
Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd_1',
       'd_2', 'd_3', 'd_4',
       ...
       'd_1905', 'd_1906', 'd_1907', 'd_1908', 'd_1909', 'd_1910', 'd_1911',
       'd_1912', 'd_1913', 'weight'],
      dtype='object', length=1920)


,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913,weight
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,3,0,1,1,1,3,0,1,1,0.000063
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0.000010
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,2,1,1,1,0,1,1,1,0.000012
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,5,4,1,0,1,3,7,2,0.000067
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,1,1,2,2,2,4,0.000024


## ML

In [22]:
# from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor


In [23]:
# train_start = 100
# train_end = 1885
# df.columns


In [24]:
# df_ml_x = df.drop(columns=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id',"weight"] + [c for c in df.columns if c.find("d_") == 0 and int(c.split("_")[1])not in range(train_start,train_end+1)])
# df_ml_y = df[[c for c in df.columns if c.find("d_") == 0 and int(c.split("_")[1]) not in range(train_end+1, train_end+28+1)] ]

In [25]:
# regressor = ExtraTreesRegressor(n_estimators=100,max_depth=50,max_features=50,random_state=42)

In [26]:
# regressor.fit(df_ml,df_ml_y)

In [27]:
# pred_df= pd.DataFrame(regressor.predict( 
#     df.drop(columns=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id',"weight"] + [c for c in df.columns if c.find("d_") == 0 and int(c.split("_")[1])not in range(train_start+28,train_end+28+1)])))

# pred_df.columns=["F_"+str(d) for d in  range(train_end+1,train_end+28+1)]

# # df+df.join(pred_df)

## step2

In [28]:
for d in range(1886, 1914):
    df["F_" + str(d)] = 0

In [29]:
print(df.shape)
# print(df.columns)
df.head()

(30490, 1948)


,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,F_1904,F_1905,F_1906,F_1907,F_1908,F_1909,F_1910,F_1911,F_1912,F_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## step 2.1 
### 计算各时期均值 没啥用

In [30]:
# df[[c for c in df.columns if c.find("d_") == 0 and int(c.split("_")[1])<1885] + ['id'] ].set_index("id").transpose()

In [31]:
complete_historical_mean_df = df[[c for c in df.columns if c.find("d_") == 0 and int(c.split("_")[1])<1885] + ['id'] ].set_index("id").transpose().mean().reset_index()

In [32]:
complete_historical_mean_df.head()

,id,0
0,HOBBIES_1_001_CA_1_validation,0.303609
1,HOBBIES_1_002_CA_1_validation,0.260085
2,HOBBIES_1_003_CA_1_validation,0.144374
3,HOBBIES_1_004_CA_1_validation,1.714968
4,HOBBIES_1_005_CA_1_validation,0.961253


In [33]:
# df[[c for c in df.columns if c.find("d_") == 0 ]].sum(axis=1).min() # not have null
df_days.sum(axis=1).min()

10

In [34]:
def find_first_non_0(s):
    assert type(s)==np.ndarray
    return (s!=0).argmax(axis=0)

In [35]:
non_0_strt_arr =[]
# hist_arr=np.array(df[[c for c in df.columns if c.find("d_") == 0 ]])
hist_arr=np.array(df_days)
for i in range(len(df)):
    non_0_strt_arr.append(find_first_non_0(hist_arr[i,:]))
non_0_strt_arr[0]
# hist_arr

901

In [36]:
# list(df[[c for c in df.columns if c.find("d_") == 0 ] + ['id']].set_index('id').transpose()['HOBBIES_1_001_CA_1_validation'])[non_0_strt_arr[0]]

In [37]:
test = list(df[[c for c in df.columns if c.find("d_") == 0 ] + ['id']].set_index('id').transpose()['HOBBIES_1_001_CA_1_validation'])

In [38]:
print(test[non_0_strt_arr[0]])
print(non_0_strt_arr[0])
print(sum(test[:non_0_strt_arr[0]] ))
print(sum(test[non_0_strt_arr[0]: ] ))

1
901
0
600


In [39]:
num_non_zero =1885-np.array(non_0_strt_arr)
num_non_zero

array([ 984, 1742,  780, ..., 1884,  946,  755])

In [40]:
non_zero_historical_mean_arr = np.array(df[[c for c in df.columns if c.find("d_") == 0 and int(c.split("_")[1])<1885] + ['id'] ].set_index("id").transpose().mean().reset_index()[0])/num_non_zero
non_zero_historical_mean_arr

array([0.00030855, 0.0001493 , 0.00018509, ..., 0.000475  , 0.00037705,
       0.00090972])

In [41]:
for d in range(1,29):
    df['F_1_'+str(1885+d)] = list(complete_historical_mean_df[0])
    df['F_2_'+str(1885+d)] = non_zero_historical_mean_arr

In [42]:
method_dict={1:"complete hsitorical mean",2:"historical mean after first non-zero"}

historical_mean_df10= df[[c for c in df.columns if c.find("d_") == 0 and int(c.split("_")[1]) in range(1876,1886) ] + ['id'] ].set_index("id").transpose().mean().reset_index()
historical_mean_df20= df[[c for c in df.columns if c.find("d_") == 0 and int(c.split("_")[1]) in range(1866,1886) ] + ['id'] ].set_index("id").transpose().mean().reset_index()
historical_mean_df30= df[[c for c in df.columns if c.find("d_") == 0 and int(c.split("_")[1]) in range(1856,1886) ] + ['id'] ].set_index("id").transpose().mean().reset_index()
historical_mean_df40= df[[c for c in df.columns if c.find("d_") == 0 and int(c.split("_")[1]) in range(1846,1886) ] + ['id'] ].set_index("id").transpose().mean().reset_index()

In [43]:
for d in range(1,29):
    df['F_3_'+str(1885+d)] = list(historical_mean_df10[0])
    df['F_4_'+str(1885+d)] = list(historical_mean_df20[0])
    df['F_5_'+str(1885+d)] = list(historical_mean_df30[0])
    df['F_6_'+str(1885+d)] = list(historical_mean_df40[0])


In [44]:
for d in range(1,29):
    df['F_7_'+str(1885+d)] = df["d_"+str(1885+d-28)]
    


In [45]:
method_dict[3] = "historical mean of recent 10 days"
method_dict[4] = "historical mean of recent 20 days"
method_dict[5] = "historical mean of recent 30 days"
method_dict[6] = "historical mean of recent 40 days"
method_dict[7] = "same as previous 28days"

In [46]:
df

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,F_7_1904,F_7_1905,F_7_1906,F_7_1907,F_7_1908,F_7_1909,F_7_1910,F_7_1911,F_7_1912,F_7_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,3,1,3,1,2,2,0,1,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,1,1,1,1
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,0,0,0,0,0,1,1,0
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,4,2,1,4,1,3,5,0,6,6
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,3,2,2,2,3,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,0,0,0,0,0,0,0,0,0,0
30486,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30487,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,2,0,0,2,0,0,0,2,0,1
30488,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,1,1,1,1,0,2,1,1,2,4


## step3 
### *agg_df 12 level*

In [47]:
agg_df = pd.DataFrame(
    df[[c for c in df.columns if c.find("d_") == 0 or c.find("F_") == 0]].sum()
    ).transpose()
agg_df

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,F_7_1904,F_7_1905,F_7_1906,F_7_1907,F_7_1908,F_7_1909,F_7_1910,F_7_1911,F_7_1912,F_7_1913
0,32631.0,31749.0,23783.0,25412.0,19146.0,29211.0,28010.0,37932.0,32736.0,25572.0,...,39652.0,46181.0,47825.0,37360.0,35475.0,34786.0,34003.0,45611.0,53863.0,46360.0


In [48]:
agg_df["level"] = 1
agg_df["weight"] = 1/12
column_order = agg_df.columns

In [49]:
level_groupings = {2: ["state_id"], 3: ["store_id"], 4: ["cat_id"], 5: ["dept_id"], 
              6: ["state_id", "cat_id"], 7: ["state_id", "dept_id"], 8: ["store_id", "cat_id"], 9: ["store_id", "dept_id"],
              10: ["item_id"], 11: ["item_id", "state_id"]}

In [50]:
df.groupby(by=level_groupings[11]).sum()

d_1  d_2  d_3  d_4  d_5  d_6  d_7  d_8  d_9  d_10  \
item_id         state_id                                                      
FOODS_1_001     CA          6    3    2    3    7    5    8    3    5     2   
                TX          0    1    2    2    0    4    0    0    4     3   
                WI          0    2    0    1    0    9    2    1    2     5   
FOODS_1_002     CA          3    3    4    4    3    3    0    2    1     1   
                TX          0    0    2    0    0    0    0    1    0     0   
...                       ...  ...  ...  ...  ...  ...  ...  ...  ...   ...   
HOUSEHOLD_2_515 TX          0    0    0    0    0    0    0    0    0     0   
                WI          0    0    0    0    0    0    0    0    0     0   
HOUSEHOLD_2_516 CA          0    2    0    1    1    1    0    2    1     2   
                TX          2    1    0    0    0    0    1    1    0     0   
                WI          0    1    0    2    0    1    1    1    0     0   

                          ...  F_7_1904  F_7_1905  F_7_1906  F_7_1907  \
item_id         state_id  ...                                           
FOODS_1_001     CA        ...         3         5         8         4   
                TX        ...         0         3         2         2   
                WI        ...         0         2         2         0   
FOODS_1_002     CA        ...         3         1         2         1   
                TX        ...         1         0         2         1   
...                       ...       ...       ...       ...       ...   
HOUSEHOLD_2_515 TX        ...         0         0         0         0   
                WI        ...         0         1         0         1   
HOUSEHOLD_2_516 CA        ...         0         1         1         0   
                TX        ...         2         0         0         1   
                WI        ...         0         1         0         0   

                          F_7_1908  F_7_1909  F_7_1910  F_7_1911  F_7_1912  \
item_id         state_id                                                     
FOODS_1_001     CA               2         4         1         2        16   
                TX               0         1         2         3         1   
                WI               0         1         0         4         1   
FOODS_1_002     CA               2         2         0         2         5   
                TX               0         0         1         2         1   
...                            ...       ...       ...       ...       ...   
HOUSEHOLD_2_515 TX               0         0         0         0         0   
                WI               0         1         0         0         1   
HOUSEHOLD_2_516 CA               0         1         1         0         3   
                TX               0         0         1         2         0   
                WI               0         0         0         0         0   

                          F_7_1913  
item_id         state_id            
FOODS_1_001     CA               1  
                TX               1  
                WI               2  
FOODS_1_002     CA               1  
                TX               0  
...                            ...  
HOUSEHOLD_2_515 TX               0  
                WI               0  
HOUSEHOLD_2_516 CA               2  
                TX               1  
                WI               0  

[9147 rows x 2138 columns]

In [51]:
for level in tqdm(level_groupings):
    temp_df = df.groupby(by=level_groupings[level]).sum().reset_index(drop=True)
    temp_df["level"] = level
    temp_df["weight"] /= 12
    agg_df = agg_df.append(temp_df[column_order])

del temp_df

100%|██████████| 10/10 [00:19<00:00,  1.91s/it]


In [52]:
print(agg_df.shape)
print(agg_df.columns)
agg_df.head(20)

(12350, 2139)
Index(['d_1', 'd_2', 'd_3', 'd_4', 'd_5', 'd_6', 'd_7', 'd_8', 'd_9', 'd_10',
       ...
       'F_7_1906', 'F_7_1907', 'F_7_1908', 'F_7_1909', 'F_7_1910', 'F_7_1911',
       'F_7_1912', 'F_7_1913', 'level', 'weight'],
      dtype='object', length=2139)


,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,F_7_1906,F_7_1907,F_7_1908,F_7_1909,F_7_1910,F_7_1911,F_7_1912,F_7_1913,level,weight
0,32631.0,31749.0,23783.0,25412.0,19146.0,29211.0,28010.0,37932.0,32736.0,25572.0,...,47825.0,37360.0,35475.0,34786.0,34003.0,45611.0,53863.0,46360.0,1,0.083333
0,14195.0,13805.0,10108.0,11047.0,9925.0,11322.0,12251.0,16610.0,14696.0,11822.0,...,19421.0,16541.0,15581.0,15141.0,15228.0,18694.0,23787.0,21296.0,2,0.036146
1,9438.0,9630.0,6778.0,7381.0,5912.0,9006.0,6226.0,9440.0,9376.0,7319.0,...,13428.0,9973.0,9826.0,10341.0,9305.0,12235.0,13595.0,13336.0,2,0.023281
2,8998.0,8314.0,6897.0,6984.0,3309.0,8883.0,9533.0,11882.0,8664.0,6431.0,...,14976.0,10846.0,10068.0,9304.0,9470.0,14682.0,16481.0,11728.0,2,0.023907
0,4337.0,4155.0,2816.0,3051.0,2630.0,3276.0,3450.0,5437.0,4340.0,3157.0,...,6197.0,4242.0,3776.0,3770.0,3970.0,4904.0,6139.0,4669.0,3,0.009092
1,3494.0,3046.0,2121.0,2324.0,1942.0,2288.0,2629.0,3729.0,2957.0,2218.0,...,2300.0,3496.0,3645.0,3322.0,3329.0,4796.0,6429.0,6296.0,3,0.008529
2,4739.0,4827.0,3785.0,4232.0,3817.0,4369.0,4703.0,5456.0,5581.0,4912.0,...,7775.0,6239.0,5703.0,5651.0,5409.0,6373.0,7978.0,7650.0,3,0.013165
3,1625.0,1777.0,1386.0,1440.0,1536.0,1389.0,1469.0,1988.0,1818.0,1535.0,...,3149.0,2564.0,2457.0,2398.0,2520.0,2621.0,3241.0,2681.0,3,0.005360
4,2556.0,2687.0,1822.0,2258.0,1694.0,2734.0,1691.0,2820.0,2887.0,2174.0,...,4133.0,2757.0,2817.0,2854.0,2637.0,3478.0,4023.0,4589.0,3,0.006599
5,3852.0,3937.0,2731.0,2954.0,2492.0,3439.0,2588.0,3772.0,3657.0,2932.0,...,4928.0,3666.0,3664.0,3900.0,3360.0,4600.0,5046.0,4759.0,3,0.008438


In [53]:
df["weight"] /= 12


In [54]:
print(df.shape)
print(df.columns)
df.head()

(30490, 2144)
Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd_1',
       'd_2', 'd_3', 'd_4',
       ...
       'F_7_1904', 'F_7_1905', 'F_7_1906', 'F_7_1907', 'F_7_1908', 'F_7_1909',
       'F_7_1910', 'F_7_1911', 'F_7_1912', 'F_7_1913'],
      dtype='object', length=2144)


,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,F_7_1904,F_7_1905,F_7_1906,F_7_1907,F_7_1908,F_7_1909,F_7_1910,F_7_1911,F_7_1912,F_7_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,3,1,3,1,2,2,0,1,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,1,1,1,1
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,0,0,0,0,0,1,1,0
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,4,2,1,4,1,3,5,0,6,6
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,3,2,2,2,3,1,0,0,0,0


In [55]:
print(df.shape[0], agg_df.shape[0], df.shape[0] + agg_df.shape[0])


30490 12350 42840


In [56]:
agg_df["weight"].sum() + df["weight"].sum()


0.9999999999999996

In [62]:
agg_df.to_csv("agg_df_12level.csv")

## step4 
### 验证算法，没啥用

In [57]:
h = 28
n = 1885
def rmsse(ground_truth, forecast, train_series, axis=1):
    # assuming input are numpy array or matrices
    assert axis == 0 or axis == 1
    assert type(ground_truth) == np.ndarray and type(forecast) == np.ndarray and type(train_series) == np.ndarray
    
    if axis == 1:
        # using axis == 1 we must guarantee these are matrices and not arrays
        assert ground_truth.shape[1] > 1 and forecast.shape[1] > 1 and train_series.shape[1] > 1
    
    numerator = ((ground_truth - forecast)**2).sum(axis=axis)
    if axis == 1:
        denominator = 1/(n-1) * ((train_series[:, 1:] - train_series[:, :-1]) ** 2).sum(axis=axis)
    else:
        denominator = 1/(n-1) * ((train_series[1:] - train_series[:-1]) ** 2).sum(axis=axis)
    return (1/h * numerator/denominator) ** 0.5

In [58]:
# train_series_cols = days_cols[:-28] # X_train
# ground_truth_cols = df_days[-28:] # y_train
# forecast_cols = df.columns[df.columns.str.contains("F_")].values # y_pred


train_series_cols = [c for c in df.columns if c.find("d_") == 0][:-28]
ground_truth_cols = [c for c in df.columns if c.find("d_") == 0][-28:]

forcast_cols_dict={}
for i in range(1,9):
    forcast_cols_dict[i] = [c for c in df.columns if c.find("F_"+str(i)+"_") == 0]


In [59]:
np.array(df[ground_truth_cols])
# np.array(df[forecast_cols])
# df[forcast_cols_dict[1]]
np.array(df[train_series_cols])

array([[0, 0, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 1, 1, 0],
       ...,
       [0, 6, 0, ..., 2, 0, 1],
       [0, 0, 0, ..., 1, 2, 4],
       [0, 0, 0, ..., 2, 0, 5]])

In [60]:
# for i in range(1,9):
#     df['rmsse_'+str(i)] = rmsse(np.array(df[ground_truth_cols]),  np.array(df[forcast_cols_dict[i]]), np.array(df[train_series_cols]))
#     agg_df["rmsse"+str(i)] = rmsse(np.array(agg_df[ground_truth_cols]),np.array(agg_df[forcast_cols_dict[i]]), np.array(agg_df[train_series_cols]))

In [63]:
# print(len(train_series_cols))
# print(len(ground_truth_cols))
# print(len(forecast_cols))

In [64]:
df

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,F_7_1904,F_7_1905,F_7_1906,F_7_1907,F_7_1908,F_7_1909,F_7_1910,F_7_1911,F_7_1912,F_7_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,3,1,3,1,2,2,0,1,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,1,1,1,1
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,0,0,0,0,0,1,1,0
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,4,2,1,4,1,3,5,0,6,6
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,3,2,2,2,3,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,0,0,0,0,0,0,0,0,0,0
30486,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30487,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,2,0,0,2,0,0,0,2,0,1
30488,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,1,1,1,1,0,2,1,1,2,4


## step5 
### 输出

In [ ]:
sample_sub = pd.read_csv('../datasets/sample_submission.csv')
print(sample_sub.shape)
sample_sub.head()

In [ ]:
sample_sub.tail()

In [ ]:
(sample_sub['id'][:len(df)] == df['id']).all()
print(df.shape)


In [ ]:
df.head()

In [ ]:
sample_sub['id'][len(df):]

In [ ]:
df.head()

In [ ]:
submit_df = df[['id']]
for i in range(1,29):
    submit_df['F'+str(i)]= df['F_7_'+str(1885+i)]

In [ ]:
submit_df2 = submit_df.copy()
submit_df2['id'] = submit_df2['id'].apply(lambda x:x.replace("validation","evaluation"))

In [ ]:
submit_df = submit_df.append(submit_df2).reset_index(drop=True)

In [ ]:
submit_df

In [ ]:
submit_df.to_csv("submission.csv",index=False)